In [165]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [166]:
xlsx = pd.ExcelFile('data/meat_statistics.xlsx')

The original excel file was designed for human readable, including merged cells for the first category (a.k.a Commerical vs. Federally Inspected below) and then individual cells for the secondary category (a.k.a row 0). 

In [167]:
raw_data = pd.read_excel(xlsx, sheet_name = 'RedMeatPoultry_Prod-Full', header = 1)
raw_data.head()

,Type 1/,Commercial 2/,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Federally inspected,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,NaN,Beef 3/,Veal 3/,Pork 3/,Lamb and mutton 3/,NaN,Total red meat 3/ 4/,Beef 3/,Veal 3/,Pork 3/,Lamb and mutton 3/,Total red meat 3/ 4/,Broilers 5/,Other chicken 5/,Turkey 5/,Total poultry 4/ 5/ 6/,Total red meat and poultry 4/,NaN
1,Jan-Sep 2022,21237.3,40.2,20075.5,98.6,NaN,41451.6,20893.5,39.2,19983.1,87.6,41003.3,34348.1,424.116,3912.59,38787.7,79791,NaN
2,Jan-Sep 2021,20842.1,38.8,20489.8,103.1,NaN,41473.9,20490.1,37.7,20388.7,90.5,41007.3,33706.1,407.63,4192.11,38397.9,79405.2,NaN
3,Sep-2022,2390.9,4.2,2259.4,10.7,NaN,4665.2,2350.5,4.1,2247.7,9.6,4611.8,4003.83,45.102,420.551,4481.28,9093.08,NaN
4,Aug-2022,2506.5,4.7,2303.3,11.7,NaN,4826.1,2466.8,4.5,2291.1,10.3,4772.8,4210.98,55.285,449.156,4727.37,9500.17,NaN


There are two types - Commerical vs. Federally Inspected. Their numbers are pretty close. I decided to use the numbers under Federally Inspected because it contains more information in terms of meat types. 

In [168]:
idx = list(raw_data.columns).index('Federally inspected')
idxs = [0]
for i in range(len(raw_data.columns)):
    if i >= idx:
        idxs.append(i)

In [169]:
raw_data = raw_data.iloc[:, idxs]
raw_data.head()

,Type 1/,Federally inspected,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,NaN,Beef 3/,Veal 3/,Pork 3/,Lamb and mutton 3/,Total red meat 3/ 4/,Broilers 5/,Other chicken 5/,Turkey 5/,Total poultry 4/ 5/ 6/,Total red meat and poultry 4/,NaN
1,Jan-Sep 2022,20893.5,39.2,19983.1,87.6,41003.3,34348.1,424.116,3912.59,38787.7,79791,NaN
2,Jan-Sep 2021,20490.1,37.7,20388.7,90.5,41007.3,33706.1,407.63,4192.11,38397.9,79405.2,NaN
3,Sep-2022,2350.5,4.1,2247.7,9.6,4611.8,4003.83,45.102,420.551,4481.28,9093.08,NaN
4,Aug-2022,2466.8,4.5,2291.1,10.3,4772.8,4210.98,55.285,449.156,4727.37,9500.17,NaN


Replace the current header with the first row (a.k.a the secondary categories) and remove the empty column

In [170]:
new_header = raw_data.iloc[0, :-1]
raw_data = raw_data.iloc[1:, :-1]
raw_data.columns = new_header
raw_data.head()

,NaN,Beef 3/,Veal 3/,Pork 3/,Lamb and mutton 3/,Total red meat 3/ 4/,Broilers 5/,Other chicken 5/,Turkey 5/,Total poultry 4/ 5/ 6/,Total red meat and poultry 4/
1,Jan-Sep 2022,20893.5,39.2,19983.1,87.6,41003.3,34348.1,424.116,3912.59,38787.7,79791
2,Jan-Sep 2021,20490.1,37.7,20388.7,90.5,41007.3,33706.1,407.63,4192.11,38397.9,79405.2
3,Sep-2022,2350.5,4.1,2247.7,9.6,4611.8,4003.83,45.102,420.551,4481.28,9093.08
4,Aug-2022,2466.8,4.5,2291.1,10.3,4772.8,4210.98,55.285,449.156,4727.37,9500.17
5,Jul-2022,2214.4,4,1960.5,8.9,4187.8,3684.82,45.709,393.913,4134.36,8322.16


Transform the header by removing space and notation, explained below
- 1/ Excludes slaughter on farms.																
- 2/ Production in federally inspected and other plants.															
- 3/ Based on packers' dressed weights.																
- 4/ Totals may not add due to rounding.																
- 5/ Ready-to-cook.																
- 6/ Includes geese, guineas, ostriches, emus, rheas, squab, and other poultry.																

In [171]:
current_header = raw_data.columns[1:] 
transformed_header = ['Month']
for i in current_header:
    word = re.search('\D+', i).group().strip().replace(' ', '_')
    transformed_header.append(word.lower())
raw_data.columns = transformed_header
raw_data.head()

,Month,beef,veal,pork,lamb_and_mutton,total_red_meat,broilers,other_chicken,turkey,total_poultry,total_red_meat_and_poultry
1,Jan-Sep 2022,20893.5,39.2,19983.1,87.6,41003.3,34348.1,424.116,3912.59,38787.7,79791
2,Jan-Sep 2021,20490.1,37.7,20388.7,90.5,41007.3,33706.1,407.63,4192.11,38397.9,79405.2
3,Sep-2022,2350.5,4.1,2247.7,9.6,4611.8,4003.83,45.102,420.551,4481.28,9093.08
4,Aug-2022,2466.8,4.5,2291.1,10.3,4772.8,4210.98,55.285,449.156,4727.37,9500.17
5,Jul-2022,2214.4,4,1960.5,8.9,4187.8,3684.82,45.709,393.913,4134.36,8322.16


Remove the rows containing additional information

In [172]:
raw_data.tail(10)

,Month,beef,veal,pork,lamb_and_mutton,total_red_meat,broilers,other_chicken,turkey,total_poultry,total_red_meat_and_poultry
1222,Feb-1921,279,24,484,39,826,NaN,NaN,NaN,NaN,NaN
1223,Jan-1921,358,27,565,44,994,NaN,NaN,NaN,NaN,NaN
1224,1/ Excludes slaughter on farms.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1225,2/ Production in federally inspected and other...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1226,3/ Based on packers' dressed weights.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1227,4/ Totals may not add due to rounding.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1228,5/ Ready-to-cook.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1229,"6/ Includes geese, guineas, ostriches, emus, r...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1230,"Source: USDA, National Agricultural Statistics...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1231,Date run: 10/26/2022 7:51:13 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
trim_data = raw_data[~raw_data['Month'].str.contains('/ |Source|Date run')]
trim_data.tail()

,Month,beef,veal,pork,lamb_and_mutton,total_red_meat,broilers,other_chicken,turkey,total_poultry,total_red_meat_and_poultry
1219,May-1921,319,31,396,37,783,NaN,NaN,NaN,NaN,NaN
1220,Apr-1921,316,29,374,41,760,NaN,NaN,NaN,NaN,NaN
1221,Mar-1921,340,31,372,45,788,NaN,NaN,NaN,NaN,NaN
1222,Feb-1921,279,24,484,39,826,NaN,NaN,NaN,NaN,NaN
1223,Jan-1921,358,27,565,44,994,NaN,NaN,NaN,NaN,NaN


Keep only the monthly data and then convert to datetime data type

In [185]:
trim_data.head()

,Month,beef,veal,pork,lamb_and_mutton,total_red_meat,broilers,other_chicken,turkey,total_poultry,total_red_meat_and_poultry
1,Jan-Sep 2022,20893.5,39.2,19983.1,87.6,41003.3,34348.1,424.116,3912.59,38787.7,79791
2,Jan-Sep 2021,20490.1,37.7,20388.7,90.5,41007.3,33706.1,407.63,4192.11,38397.9,79405.2
3,Sep-2022,2350.5,4.1,2247.7,9.6,4611.8,4003.83,45.102,420.551,4481.28,9093.08
4,Aug-2022,2466.8,4.5,2291.1,10.3,4772.8,4210.98,55.285,449.156,4727.37,9500.17
5,Jul-2022,2214.4,4,1960.5,8.9,4187.8,3684.82,45.709,393.913,4134.36,8322.16


In [184]:
meat_prod = trim_data.copy()
meat_prod = meat_prod[meat_prod['Month'].str.contains(r'\w{3}-\d{4}', regex=True)]
meat_prod.head()

,Month,beef,veal,pork,lamb_and_mutton,total_red_meat,broilers,other_chicken,turkey,total_poultry,total_red_meat_and_poultry
3,Sep-2022,2350.5,4.1,2247.7,9.6,4611.8,4003.83,45.102,420.551,4481.28,9093.08
4,Aug-2022,2466.8,4.5,2291.1,10.3,4772.8,4210.98,55.285,449.156,4727.37,9500.17
5,Jul-2022,2214.4,4,1960.5,8.9,4187.8,3684.82,45.709,393.913,4134.36,8322.16
6,Jun-2022,2415.1,4.5,2245.1,10.2,4674.8,3934.88,52.399,442.265,4440.92,9115.72
7,May-2022,2250.1,4.3,2171.9,9.6,4435.9,3778.86,47.351,412.789,4249.4,8685.3


In [188]:
meat_prod['Month'] = meat_prod['Month'].apply(lambda x: datetime.strptime(x, '%b-%Y'))
meat_prod.head()

,Month,beef,veal,pork,lamb_and_mutton,total_red_meat,broilers,other_chicken,turkey,total_poultry,total_red_meat_and_poultry
3,2022-09-01,2350.5,4.1,2247.7,9.6,4611.8,4003.83,45.102,420.551,4481.28,9093.08
4,2022-08-01,2466.8,4.5,2291.1,10.3,4772.8,4210.98,55.285,449.156,4727.37,9500.17
5,2022-07-01,2214.4,4,1960.5,8.9,4187.8,3684.82,45.709,393.913,4134.36,8322.16
6,2022-06-01,2415.1,4.5,2245.1,10.2,4674.8,3934.88,52.399,442.265,4440.92,9115.72
7,2022-05-01,2250.1,4.3,2171.9,9.6,4435.9,3778.86,47.351,412.789,4249.4,8685.3


In [189]:
meat_prod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1221 entries, 3 to 1223
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Month                       1221 non-null   datetime64[ns]
 1   beef                        985 non-null    object        
 2   veal                        985 non-null    object        
 3   pork                        985 non-null    object        
 4   lamb_and_mutton             985 non-null    object        
 5   total_red_meat              985 non-null    object        
 6   broilers                    753 non-null    object        
 7   other_chicken               261 non-null    object        
 8   turkey                      753 non-null    object        
 9   total_poultry               753 non-null    object        
 10  total_red_meat_and_poultry  541 non-null    object        
dtypes: datetime64[ns](1), object(10)
memory usage: 114.5+ KB